<a href="https://colab.research.google.com/github/s-a-malik/zero-shot/blob/main/data_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!tar -xf "/content/drive/My Drive/NLP project/Dataset/data.gz" -C /content/


gzip: stdin: No space left on device
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [ ]:
!cp "/content/drive/My Drive/NLP project/Dataset/train.json" /content/zanim

In [ ]:
!cp "/content/drive/My Drive/NLP project/Dataset/train/03430_Animalia_Chordata_Aves_Charadriiformes_Laridae_Rynchops_niger/ef84ad6c-7404-43b2-a131-a18d071b00f1.jpg" "/content/zanim/train/03430_Animalia_Chordata_Aves_Charadriiformes_Laridae_Rynchops_niger/ef84ad6c-7404-43b2-a131-a18d071b00f1.jpg"

In [ ]:
!cp "/content/drive/My Drive/NLP project/Dataset/data.gz" /content/
!tar -xf data.gz
!cp "/content/drive/My Drive/NLP project/Dataset/train.json" /content/zanim
!rm -rf data.gz

cp: error reading '/content/drive/My Drive/NLP project/Dataset/data.gz': No space left on device
cp: cannot create regular file '/content/zanim/': Not a directory


In [ ]:
!zip -r "/content/drive/My Drive/NLP project/Dataset/dataset.zip" "/content/drive/My Drive/NLP project/Dataset/"

Streaming output truncated to the last 5000 lines.
  adding: content/drive/My Drive/NLP project/Dataset/train/04371_Animalia_Chordata_Aves_Pelecaniformes_Ardeidae_Butorides_virescens/24c7df30-ab32-4d96-b0db-05cfe6c78264.jpg (deflated 0%)
  adding: content/drive/My Drive/NLP project/Dataset/train/04371_Animalia_Chordata_Aves_Pelecaniformes_Ardeidae_Butorides_virescens/8e2129fe-d2aa-41ab-bcc2-57271d0e134d.jpg (deflated 0%)
  adding: content/drive/My Drive/NLP project/Dataset/train/04371_Animalia_Chordata_Aves_Pelecaniformes_Ardeidae_Butorides_virescens/cf4898a6-5b62-443b-86de-6ace19d65b49.jpg (deflated 3%)
  adding: content/drive/My Drive/NLP project/Dataset/train/04371_Animalia_Chordata_Aves_Pelecaniformes_Ardeidae_Butorides_virescens/7627d19c-5420-473d-9ba5-0cc575e4c0bf.jpg (deflated 3%)
  adding: content/drive/My Drive/NLP project/Dataset/train/04371_Animalia_Chordata_Aves_Pelecaniformes_Ardeidae_Butorides_virescens/d1f5e6db-770e-45a8-9134-0723088f40d2.jpg (deflated 0%)
  adding: cont

In [ ]:
# !pip install transformers
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# !mkdir /content/data
# !cp -R "/content/drive/My Drive/NLP project/Dataset/" /content/data/
!rsync --info=progress2 -r "/content/drive/My Drive/NLP project/Dataset/" /content/data/

  7,425,059,427  98%  842.77kB/s    2:23:23 (xfr#30090, ir-chk=1310/32532)
rsync error: received SIGINT, SIGTERM, or SIGHUP (code 20) at rsync.c(644) [sender=3.1.2]
rsync error: received SIGINT, SIGTERM, or SIGHUP (code 20) at io.c(513) [generator=3.1.2]


In [ ]:
import torchvision.models as models
from torch import nn
import torch

resnet = models.resnet152(pretrained=True)
class Identity(nn.Module):
  def __init__(self):
    super().__init__()
  
  def forward(self, x):
    return x

resnet.fc = Identity()
resnet.to(device)
pass

In [ ]:
import torch.utils.data as data
import json
import os
from PIL import Image
import subprocess
# from transformers import BertTokenizer, BertModel
from typing import List
from torchvision.transforms import Compose
from torchvision import transforms

class DefaultTransform(Compose):

  def __init__(self, image_size: int = 256, channel_means: List[float] = [0.485, 0.456, 0.406], channel_stds: List[float] = [0.229, 0.224, 0.225]):
    super().__init__([
          transforms.Resize(image_size),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize(channel_means, channel_stds)
    ])

class Zanim(data.Dataset):

  def __init__(self, root: str, json_path: str, device: str, image_transforms: Compose = DefaultTransform()):
    """
    :param root: the path to the root directory of the dataset
    :param json_path: the path to the json file containing the annotations
    :param transforms: a composed set of transforms
    """
    if not(root in json_path):
        json_path = os.path.join(root, json_path)
    self.root = root
    self.transforms = image_transforms
    with open(json_path) as annotations:
      annotations = json.load(annotations)

    # self.tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
    # self.tokenizer.to(device)
    # self.bert_model = BertModel.from_pretrained("bert-large-uncased")
    # self.bert_model.to(device)

    self.ids = [i['id'] for i in annotations['images']]
    self.image_files = [i['file_name'] for i in annotations['images']]
    self.species_id = [annotations['annotations'][i['id']]['category_id'] for i in annotations['images']]
    self.species_name = [annotations['categories'][i]['name'] for i in self.species_id]
    self.descriptions = [annotations['categories'][index]['description'] for index in self.species_id]
    
    missing = 0
    for i in self.image_files:
      if not(os.path.exists(os.path.join(root, i))):
        # print(i)
        # break
        pipes = subprocess.Popen(["cp", f'/content/drive/My Drive/NLP project/Dataset/{i}', f'/content/zanim/{i}'], stderr=subprocess.PIPE)
        _, err = pipes.communicate()
        if pipes.returncode != 0:
          print(f"Error: {err.strip()}")
          break
        missing += 1
    print(f'MISSING: {missing}')


  def __len__(self):
    return len(self.ids)

  def __getitem__(self, index):
    file_path = os.path.join(self.root, self.image_files[index])
    try:
      image = Image.open(file_path)
    except OSError:
      print(f"Missing/Truncated image: {self.image_files[index]}")
      image = torch.zeros((250,250))
    if self.transforms:
      image = self.transforms(image)

    # image = image.to(device)
    description = self.descriptions[index] 
    # description = description.to(device)
    # tokens = tokenizer(description, return_tensors="pt")
    # species_id = self.species_id[index]
    # tokens = self.tokenizer(description, return_tensors="pt")
    # embedding = self.bert_model(**tokens)
    return self.ids[index], image, description, file_path
    # return (image, embedding), species_id, {'description': description, 
                                            # 'species': self.species_name[index],
                                            # 'file path': file_path}


In [ ]:
# z = Zanim(root="/content/drive/My Drive/NLP project/Dataset", json_path="train.json", device=device, image_transforms=DefaultTransform())
z = Zanim(root="/content/zanim", json_path="train.json", device=device, image_transforms=DefaultTransform())

MISSING: 0


In [ ]:
for i in range(115712, 115712+1024):
  try:
    z.__getitem__(i)
  except Exception:
    print(i)

116454


In [ ]:
z.__getitem__(116454)

(116454, tensor([[[ 0.4508,  0.4851,  0.4508,  ...,  0.0741, -0.0287, -0.0629],
          [ 0.6049,  0.6049,  0.5707,  ..., -0.1314, -0.1999, -0.2513],
          [ 0.7077,  0.6392,  0.5707,  ..., -0.1999, -0.2342, -0.2856],
          ...,
          [-0.3198, -0.3369, -0.2171,  ...,  0.4851,  0.5193,  0.4851],
          [-0.3198, -0.3712, -0.2513,  ...,  0.5193,  0.5193,  0.4679],
          [-0.1657, -0.3027, -0.2513,  ...,  0.4508,  0.3994,  0.4337]],
 
         [[ 0.2577,  0.2927,  0.2577,  ..., -0.2150, -0.3200, -0.3550],
          [ 0.4328,  0.4328,  0.3978,  ..., -0.4251, -0.5126, -0.5476],
          [ 0.5553,  0.4853,  0.4328,  ..., -0.4951, -0.5476, -0.6001],
          ...,
          [-0.6702, -0.6877, -0.5476,  ...,  0.1702,  0.2052,  0.1702],
          [-0.6527, -0.7052, -0.6001,  ...,  0.2227,  0.2227,  0.1877],
          [-0.4776, -0.6352, -0.5651,  ...,  0.1702,  0.1176,  0.1527]],
 
         [[-0.0964, -0.0615, -0.0790,  ..., -0.5321, -0.6367, -0.6715],
          [ 0.0779, 

In [43]:
z.image_files[116454]

'train/03430_Animalia_Chordata_Aves_Charadriiformes_Laridae_Rynchops_niger/ef84ad6c-7404-43b2-a131-a18d071b00f1.jpg'

In [ ]:
# species_written = set()
# from tqdm.notebook import tqdm
# tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
# # tokenizer.to(device)
# bert_model = BertModel.from_pretrained("bert-large-uncased")
# bert_model.to(device)
# pass

In [45]:
import h5py
import numpy as np
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
h5_file = h5py.File(os.path.join(z.root, "image-embed-5.hdf5"), 'w')
images = h5_file.create_dataset('images', shape=(len(z), 2048), dtype=np.float32, fillvalue=0)
for ids, ims, _, file_path in tqdm(DataLoader(z, batch_size=512)):
  ims = ims.to(device)
  with torch.no_grad():
    # tokens = tokenizer(description, return_tensors="pt", padding=True, truncation=True)
    # tokens = tokens.to(device)
    # embedding = bert_model(**tokens)
    # file_path = dic['file path']
    im_em = resnet(ims)
  # em_path = [f.split(".")[0].replace("train", "embedded-hdf5") for f in file_path]
  # paths = [f.split("/") for f in em_path]
  # for p in paths:
  #   dir = os.sep + os.path.join(*p[:-1]) + os.sep
  #   if not(os.path.exists(dir)):
  #     os.mkdir(dir)
  # im_em_paths =  [f + ".pt" for f in em_path]
  # desc_em_paths = [os.sep + os.path.join(*p[:-1],p[-2]+".pt") for p in paths]
  # print(embedding)
  # print(embedding[0].shape)
  # print(embedding[1].shape)
  # break
  # for i,d in enumerate(desc_em_paths):
    # if not(d in species_written):
      # species_written.add(d)
      # torch.save(embedding.pooler_output[i], d)

  for i in range(im_em.shape[0]):
    images[ids[i]] = im_em[i].cpu().numpy()
    # torch.save(im_em[i], p)
  del ims
  del im_em
  # del embedding
  # del tokens
  torch.cuda.empty_cache()
h5_file.close()

In [46]:
!cp "/content/zanim/image-embed-5.hdf5" "/content/drive/My Drive/NLP project/Dataset/image-embedding.hdf5"

In [ ]:
h5_file.close()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
(i,e),s,d = z.__getitem__(15)

In [ ]:
print(d)
i

{'description': 'Black-and-white warblers are small songbirds that breed in northern and eastern parts of North America. True to their name, these birds are black and white in color. Both sexes have black and white crowns with a white eyebrow, black streaking on a white belly, black wings with two white wing bars, a black tail, a black-and-white streaked back, streaky undertail coverts, and grey-black legs and feet. Breeding males have a black-and-white streaked throat and black cheek, while females have a grey cheek and a white-cream colored throat and sides. Summer male black-and-white warblers are boldly streaked in black and white and have been described as flying humbugs. Each wing is black with two white wing bars. Female and juvenile plumages are similar but duller and less streaky than males.', 'species': 'Mniotilta varia'}


tensor([[[-0.9020, -0.8863, -0.7922,  ..., -0.8392, -1.0275, -1.0745],
         [-0.8235, -0.8078, -0.6196,  ..., -0.8078, -0.9961, -1.0745],
         [-0.8392, -0.7922, -0.5569,  ..., -0.7922, -0.9647, -1.0275],
         ...,
         [-0.3843, -0.3216,  0.4471,  ...,  0.5412,  0.6196,  0.6824],
         [-0.4784, -0.4627,  0.1020,  ...,  0.5098,  0.6196,  0.6824],
         [-0.5569, -0.6353, -0.2431,  ...,  0.4784,  0.6039,  0.6667]],

        [[-1.0118, -1.0275, -0.9020,  ..., -1.0745, -1.2314, -1.2784],
         [-0.9490, -0.9176, -0.6667,  ..., -1.0275, -1.1843, -1.2627],
         [-0.9961, -0.8863, -0.5725,  ..., -0.9961, -1.1686, -1.2000],
         ...,
         [-0.5255, -0.4471,  0.4000,  ...,  0.3216,  0.3686,  0.4157],
         [-0.6667, -0.6196,  0.0549,  ...,  0.2902,  0.3529,  0.4157],
         [-0.6980, -0.8078, -0.3686,  ...,  0.2588,  0.3529,  0.4000]],

        [[-1.1529, -1.1373, -1.0431,  ..., -1.3255, -1.4824, -1.5294],
         [-1.0588, -1.0588, -0.7922,  ..., -1

In [ ]:
print("Species names")
print("\n".join(z.species_name[:10]))
print("\nSpecies ID")
print(z.species_id[:10])
print("\nDescriptions")
print("\n".join(z.descriptions[:10]))
print("\nImage files examples")
print("\n".join(z.image_files[:10]))

Species names
Marmota flaviventris
Dipsosaurus dorsalis
Gopherus agassizii
Grampus griseus
Sterna paradisaea
Fratercula arctica
Cistothorus palustris
Falco mexicanus
Crocodylus acutus
Lanius ludovicianus

Species ID
[524, 607, 665, 388, 100, 84, 242, 138, 545, 205]

Descriptions
Yellow-bellied marmots are rodents similar to squirrels, but bigger. They have grizzled brownish fur and a yellow belly, with a whitish spot between their eyes. Their ears are small and round and they have a short white muzzle with a black nose. Their bodies are heavy-set and they have short legs and a reddish-brown furry tail.
The Desert iguana is one of the most common lizards of the Sonoran and Mojave deserts of the southwestern United States and northwestern Mexico. It is pale gray-tan to cream in color with a light brown reticulated pattern on its back and sides. Down the center of the back is a row of slightly-enlarged, keeled dorsal scales that become slightly larger as you move down the back. The reticu

# Generating SSH key for using drive and github

**PUT YOUR EMAIL BELOW**

In [ ]:
!ssh-keygen -t ed25519 -C "youmed.tech@gmail.com"

Generating public/private ed25519 key pair.
Enter file in which to save the key (/root/.ssh/id_ed25519): /content/drive/My Drive/ssh/id_ed25519
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /content/drive/My Drive/ssh/id_ed25519.
Your public key has been saved in /content/drive/My Drive/ssh/id_ed25519.pub.
The key fingerprint is:
SHA256:gj/J3Z3Uwyr2H/bwCbhphn+OCtJ3aXon4hoj9MskOUI youmed.tech@gmail.com
The key's randomart image is:
+--[ED25519 256]--+
|                 |
|                 |
|                 |
|     .       o   |
|   E... S   . +  |
|  . .o++ . ooo . |
|   . *=B..==+.+  |
|    . B.*++B++.=.|
|       +o+B=*o..o|
+----[SHA256]-----+


**CREATE A FOLDER SSH IN YOUR GDRIVE**

In [ ]:
!rm -rf /etc/ssh/ssh_config
!touch /etc/ssh/ssh_config
!echo "Host *" >> /etc/ssh/ssh_config
!echo " AddKeysToAgent yes" >> /etc/ssh/ssh_config
!echo " IgnoreUnknown UseKeychain" >> /etc/ssh/ssh_config
!echo " IdentityFile \"/content/drive/My Drive/ssh/id_ed25519\"" >> /etc/ssh/ssh_config

In [ ]:
!eval "$(ssh-agent -s)" && ssh-add -k /content/drive/My\ Drive/ssh/id_ed25519

Agent pid 1697
Identity added: /content/drive/My Drive/ssh/id_ed25519 (youmed.tech@gmail.com)


**COPY THE KEY FROM YOUR GDRIVE AND ADD TO YOUR GITHUB ACCOUNT**

In [ ]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
!git clone git@github.com:s-a-malik/zero-shot.git

# github.com:22 SSH-2.0-babeld-8514a139
Cloning into 'zero-shot'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 7), reused 14 (delta 2), pack-reused 0
Receiving objects: 100% (26/26), 679.75 KiB | 2.57 MiB/s, done.
Resolving deltas: 100% (7/7), done.
